In [32]:
import javalang
from codegraph.java_parser_copy import _anno_details, _extract_http_basic, _anno_kv


src = r'''
package com.example.demo.controllers;
import org.springframework.stereotype.Controller;
import org.springframework.web.bind.annotation.*;
@Controller
public class ProductController {
  @PostMapping("/addingProduct")
  public String addProduct(@ModelAttribute Product product) { return "ok"; }
  @GetMapping("/updatingProduct/{productId}")
  public String updateProduct(@ModelAttribute Product product, @PathVariable("productId") int id) { return "ok"; }
  @GetMapping("/deleteProduct/{productId}")
  public String delete(@PathVariable("productId") int id) { return "ok"; }
}
'''
tree = javalang.parse.parse(src)
for t in tree.types:
    for m in t.methods:
        md = [_anno_details(a) for a in (m.annotations or [])]
        print(m.name, md, _extract_http_basic(md))


addProduct [{'name': '@PostMapping', 'full': '@PostMapping(/addingProduct)', 'args': {'value_list': ['/addingProduct'], 'value': '/addingProduct'}}] {'methods': ['POST'], 'paths': ['/addingProduct'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
updateProduct [{'name': '@GetMapping', 'full': '@GetMapping(/updatingProduct/{productId})', 'args': {'value_list': ['/updatingProduct/{productId}'], 'value': '/updatingProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/updatingProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
delete [{'name': '@GetMapping', 'full': '@GetMapping(/deleteProduct/{productId})', 'args': {'value_list': ['/deleteProduct/{productId}'], 'value': '/deleteProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/deleteProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}


In [2]:
from __future__ import annotations
from typing import List, Tuple, Dict, Any, Optional

import os

# Base Tree-sitter bindings
from tree_sitter import Language, Parser


# ============================================================
# Signature / runtime banner
# ============================================================
SIGNATURE = "java-parser-treesitter:ast-http-v3 (bindings-only, no tree_sitter_languages)"
print(f"[CodeGraph] Loaded {SIGNATURE}")

# ============================================================
# How we obtain a Java Language
# Tries, in order:
#   1) tree_sitter_java (preferred)
#   2) tree_sitter_pack (if available)
#   3) prebuilt .so/.dll path via TREE_SITTER_JAVA_LIB
#   4) optional build from vendor/tree-sitter-java (if present)
# ============================================================

def _get_java_language() -> Language:
    # 1) tree_sitter_java package
    try:
        # Several variants exist. We try common ones.
        import tree_sitter_java  # type: ignore
        # Some dists expose LANGUAGE directly
        lang = getattr(tree_sitter_java, "LANGUAGE", None)
        if isinstance(lang, Language):
            return lang
        # Some expose a factory function
        for candidate in ("language", "get_language", "java"):
            fn = getattr(tree_sitter_java, candidate, None)
            if callable(fn):
                lang = fn()
                if isinstance(lang, Language):
                    return lang
        # Some expose a path to buildable grammar sources; try build if vendor not needed
    except Exception:
        pass

    # 2) tree_sitter_pack (bundle with get_language)
    try:
        import tree_sitter_pack  # type: ignore
        get_language = getattr(tree_sitter_pack, "get_language", None)
        if callable(get_language):
            lang = get_language("java")
            if isinstance(lang, Language):
                return lang
    except Exception:
        pass

    # 3) Prebuilt shared lib via env var
    lib_path = os.getenv("TREE_SITTER_JAVA_LIB")
    if lib_path and os.path.exists(lib_path):
        try:
            return Language(lib_path, "java")
        except Exception as e:
            print("[CodeGraph] Failed to load TREE_SITTER_JAVA_LIB:", e)

    # 4) Build from vendor checkout (optional)
    vendor_dir = os.path.join(os.path.dirname(__file__), "vendor", "tree-sitter-java")
    if os.path.isdir(vendor_dir):
        try:
            build_dir = os.path.join("/Users/saichaitanyadarla/Downloads/codegraph_inline_full", "vendor", "build")
            os.makedirs("build_dir", exist_ok=True)
            lib_file = os.path.join(build_dir, "java-languages.so")
            # Build only once
            if not os.path.exists(lib_file):
                Language.build_library(
                    lib_file,
                    [vendor_dir],  # expects parser.c and scanner.c in src/
                )
            return Language(lib_file, "java")
        except Exception as e:
            print("[CodeGraph] Vendor build failed:", e)

    raise RuntimeError(
        "Unable to obtain Tree-sitter Java Language. Install one of: "
        "(a) tree-sitter-java, (b) tree-sitter-pack, (c) set TREE_SITTER_JAVA_LIB, or (d) vendor grammar."
    )

# Initialize the Parser with whichever Language we resolved
_JAVA_LANGUAGE = _get_java_language()
_PARSER = Parser()
_PARSER.set_language(_JAVA_LANGUAGE)

# ============================================================
# HTTP mapping registry (centralized)
# ============================================================
_HTTP_MAP = {
    "@GetMapping": "GET",
    "@PostMapping": "POST",
    "@PutMapping": "PUT",
    "@DeleteMapping": "DELETE",
    "@PatchMapping": "PATCH",
}
_REQUEST_METHOD_PREFIX = "RequestMethod."



[CodeGraph] Loaded java-parser-treesitter:ast-http-v3 (bindings-only, no tree_sitter_languages)


NameError: name '__file__' is not defined

In [ ]:
import javalang
from codegraph.java_parser_copy import _anno_details, _extract_http_basic, _anno_kv


src = r'''
package com.example.demo.controllers;
import org.springframework.stereotype.Controller;
import org.springframework.web.bind.annotation.*;
@Controller
public class ProductController {
  @PostMapping("/addingProduct")
  public String addProduct(@ModelAttribute Product product) { return "ok"; }
  @GetMapping("/updatingProduct/{productId}")
  public String updateProduct(@ModelAttribute Product product, @PathVariable("productId") int id) { return "ok"; }
  @GetMapping("/deleteProduct/{productId}")
  public String delete(@PathVariable("productId") int id) { return "ok"; }
}
'''
tree = javalang.parse.parse(src)
for t in tree.types:
    for m in t.methods:
        md = [_anno_details(a) for a in (m.annotations or [])]
        print(m.name, md, _extract_http_basic(md))


addProduct [{'name': '@PostMapping', 'full': '@PostMapping(/addingProduct)', 'args': {'value_list': ['/addingProduct'], 'value': '/addingProduct'}}] {'methods': ['POST'], 'paths': ['/addingProduct'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
updateProduct [{'name': '@GetMapping', 'full': '@GetMapping(/updatingProduct/{productId})', 'args': {'value_list': ['/updatingProduct/{productId}'], 'value': '/updatingProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/updatingProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
delete [{'name': '@GetMapping', 'full': '@GetMapping(/deleteProduct/{productId})', 'args': {'value_list': ['/deleteProduct/{productId}'], 'value': '/deleteProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/deleteProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}


In [ ]:
import javalang
from codegraph.java_parser_copy import _anno_details, _extract_http_basic, _anno_kv


src = r'''
package com.example.demo.controllers;
import org.springframework.stereotype.Controller;
import org.springframework.web.bind.annotation.*;
@Controller
public class ProductController {
  @PostMapping("/addingProduct")
  public String addProduct(@ModelAttribute Product product) { return "ok"; }
  @GetMapping("/updatingProduct/{productId}")
  public String updateProduct(@ModelAttribute Product product, @PathVariable("productId") int id) { return "ok"; }
  @GetMapping("/deleteProduct/{productId}")
  public String delete(@PathVariable("productId") int id) { return "ok"; }
}
'''
tree = javalang.parse.parse(src)
for t in tree.types:
    for m in t.methods:
        md = [_anno_details(a) for a in (m.annotations or [])]
        print(m.name, md, _extract_http_basic(md))


addProduct [{'name': '@PostMapping', 'full': '@PostMapping(/addingProduct)', 'args': {'value_list': ['/addingProduct'], 'value': '/addingProduct'}}] {'methods': ['POST'], 'paths': ['/addingProduct'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
updateProduct [{'name': '@GetMapping', 'full': '@GetMapping(/updatingProduct/{productId})', 'args': {'value_list': ['/updatingProduct/{productId}'], 'value': '/updatingProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/updatingProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
delete [{'name': '@GetMapping', 'full': '@GetMapping(/deleteProduct/{productId})', 'args': {'value_list': ['/deleteProduct/{productId}'], 'value': '/deleteProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/deleteProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}


In [ ]:
import javalang
from codegraph.java_parser_copy import _anno_details, _extract_http_basic, _anno_kv


src = r'''
package com.example.demo.controllers;
import org.springframework.stereotype.Controller;
import org.springframework.web.bind.annotation.*;
@Controller
public class ProductController {
  @PostMapping("/addingProduct")
  public String addProduct(@ModelAttribute Product product) { return "ok"; }
  @GetMapping("/updatingProduct/{productId}")
  public String updateProduct(@ModelAttribute Product product, @PathVariable("productId") int id) { return "ok"; }
  @GetMapping("/deleteProduct/{productId}")
  public String delete(@PathVariable("productId") int id) { return "ok"; }
}
'''
tree = javalang.parse.parse(src)
for t in tree.types:
    for m in t.methods:
        md = [_anno_details(a) for a in (m.annotations or [])]
        print(m.name, md, _extract_http_basic(md))


addProduct [{'name': '@PostMapping', 'full': '@PostMapping(/addingProduct)', 'args': {'value_list': ['/addingProduct'], 'value': '/addingProduct'}}] {'methods': ['POST'], 'paths': ['/addingProduct'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
updateProduct [{'name': '@GetMapping', 'full': '@GetMapping(/updatingProduct/{productId})', 'args': {'value_list': ['/updatingProduct/{productId}'], 'value': '/updatingProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/updatingProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
delete [{'name': '@GetMapping', 'full': '@GetMapping(/deleteProduct/{productId})', 'args': {'value_list': ['/deleteProduct/{productId}'], 'value': '/deleteProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/deleteProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}


In [24]:
import javalang, sys
src = open("/Users/saichaitanyadarla/Documents/java/FoodFrenzy/src/main/java/com/example/demo/controllers/ProductController.java").read()
tree = javalang.parse.parse(src)

for t in tree.types:
    print("Type:", t.name)
    for m in getattr(t, "methods", []):
        print("  Method:", m.name)
        for a in (m.annotations or []):
            print("@" + getattr(a, "name", "?"),
                  "member=", getattr(a, "member", None),
                  "value=", getattr(a, "value", None),
                  "element=", getattr(a, "element", None))


Type: ProductController
  Method: addProduct
@PostMapping member= None value= None element= Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="/addingProduct")
  Method: updateProduct
@GetMapping member= None value= None element= Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="/updatingProduct/{productId}")
  Method: delete
@GetMapping member= None value= None element= Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="/deleteProduct/{productId}")
